<a href="https://colab.research.google.com/github/ilkzeyden/text-to-flowchart/blob/main/BERT_Multi_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------
# 1. Adım: Verileri Yükleme
# -----------------------------
train_file = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'
valid_file = r'/content/drive/My Drive/akışlar-test-verileri.xlsx'

df_train = pd.read_excel(train_file)
df_valid = pd.read_excel(valid_file)

# Adım Türüne göre Koşul Label'ı belirle (Koşul varsa 1, yoksa 0)
df_train['Koşul Label'] = df_train['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)
df_valid['Koşul Label'] = df_valid['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)

# Boş olan Evet/Hayır durumlarını -1 yap (Eksik değerler için)
df_train['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_train['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

df_valid['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_valid['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

# Eksik değerleri içermeyenleri filtrele
df_train = df_train[(df_train['Evet Durumu (Sonraki Adım)'] != -1) & (df_train['Hayır Durumu (Sonraki Adım)'] != -1)]
df_valid = df_valid[(df_valid['Evet Durumu (Sonraki Adım)'] != -1) & (df_valid['Hayır Durumu (Sonraki Adım)'] != -1)]

# -----------------------------
# 2. BERT Dataset Sınıfı
# -----------------------------
class BertMultiTaskDataset(Dataset):
    def __init__(self, texts, labels_task1, labels_task2, labels_task3, labels_task4, tokenizer, max_length=128):
        self.texts = texts
        self.labels_task1 = labels_task1
        self.labels_task2 = labels_task2
        self.labels_task3 = labels_task3
        self.labels_task4 = labels_task4
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label_task1 = self.labels_task1[idx]
        label_task2 = self.labels_task2[idx]
        label_task3 = self.labels_task3[idx]
        label_task4 = self.labels_task4[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label_task1': torch.tensor(label_task1, dtype=torch.long),
            'label_task2': torch.tensor(label_task2, dtype=torch.float),
            'label_task3': torch.tensor(label_task3, dtype=torch.long),
            'label_task4': torch.tensor(label_task4, dtype=torch.long),
        }

# -----------------------------
# 3. Model Tanımı
# -----------------------------
class BertMultiTaskModel(nn.Module):
    def __init__(self, bert_model_name, num_labels_task1):
        super(BertMultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.task1_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task1)
        self.task2_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.task3_classifier = nn.Linear(self.bert.config.hidden_size, 10)
        self.task4_classifier = nn.Linear(self.bert.config.hidden_size, 10)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        task1_logits = self.task1_classifier(pooled_output)
        task2_logits = self.task2_classifier(pooled_output).squeeze(-1)
        task3_logits = self.task3_classifier(pooled_output)
        task4_logits = self.task4_classifier(pooled_output)

        return task1_logits, task2_logits, task3_logits, task4_logits

# -----------------------------
# 4. Model Hazırlığı
# -----------------------------
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
num_labels_task1 = len(df_train['Adım Türü Label'].unique())

train_dataset = BertMultiTaskDataset(df_train['Adım Açıklaması'].tolist(), df_train['Adım Türü Label'].values, df_train['Koşul Label'].values, df_train['Evet Durumu (Sonraki Adım)'].values, df_train['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)
valid_dataset = BertMultiTaskDataset(df_valid['Adım Açıklaması'].tolist(), df_valid['Adım Türü Label'].values, df_valid['Koşul Label'].values, df_valid['Evet Durumu (Sonraki Adım)'].values, df_valid['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# -----------------------------
# 5. Model Eğitimi
# -----------------------------
model = BertMultiTaskModel("dbmdz/bert-base-turkish-cased", num_labels_task1)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion_task1 = nn.CrossEntropyLoss()
criterion_task2 = nn.BCEWithLogitsLoss()
criterion_task3 = nn.CrossEntropyLoss()
criterion_task4 = nn.CrossEntropyLoss()

num_epochs = 6
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])
        loss = criterion_task1(task1_logits, batch['label_task1']) + criterion_task2(task2_logits, batch['label_task2']) + criterion_task3(task3_logits, batch['label_task3']) + criterion_task4(task4_logits, batch['label_task4'])
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} tamamlandı!")

import torch.nn as nn

# Modelin çıktılarındaki sınıf sayısını doğrula
num_labels_task1 = df_train['Adım Türü Label'].nunique()
num_labels_task3 = df_train['Evet Durumu (Sonraki Adım)'].nunique()
num_labels_task4 = df_train['Hayır Durumu (Sonraki Adım)'].nunique()

# Modelin tanımı güncellendi
self.task1_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task1)
self.task3_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task3)
self.task4_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task4)

# Loss fonksiyonları güncellendi
criterion_task1 = nn.CrossEntropyLoss()
criterion_task3 = nn.CrossEntropyLoss(ignore_index=-1)  # -1 değerlerini yok say
criterion_task4 = nn.CrossEntropyLoss(ignore_index=-1)

# Eğitim döngüsü içinde
optimizer.zero_grad()
task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

# Hata kaynağını kontrol et
max_label_task1 = batch['label_task1'].max().item()
max_label_task3 = batch['label_task3'].max().item()
max_label_task4 = batch['label_task4'].max().item()

if max_label_task1 >= num_labels_task1 or max_label_task3 >= num_labels_task3 or max_label_task4 >= num_labels_task4:
    print(f"UYARI: Hedef etiketlerden biri modelin sınıf sayısından büyük! "
          f"Task1: {max_label_task1}/{num_labels_task1}, "
          f"Task3: {max_label_task3}/{num_labels_task3}, "
          f"Task4: {max_label_task4}/{num_labels_task4}")

# Güncellenmiş loss hesaplama
loss = (
    criterion_task1(task1_logits, batch['label_task1']) +
    criterion_task2(task2_logits, batch['label_task2']) +
    criterion_task3(task3_logits, batch['label_task3']) +
    criterion_task4(task4_logits, batch['label_task4'])
)
loss.backward()
optimizer.step()

# -----------------------------
# 6. Model Değerlendirme
# -----------------------------
model.eval()
predictions_task1, predictions_task2 = [], []
with torch.no_grad():
    for batch in valid_loader:
        task1_logits, task2_logits = model(batch['input_ids'], batch['attention_mask'])

        preds_task1 = torch.argmax(task1_logits, dim=1).tolist()
        preds_task2 = (torch.sigmoid(task2_logits) > 0.5).int().tolist()

        predictions_task1.extend(preds_task1)
        predictions_task2.extend(preds_task2)

print("Task 1 (Adım Türü) Accuracy:", accuracy_score(bert_valid_y1, predictions_task1))
print("Task 1 (Adım Türü) Classification Report:")
print(classification_report(bert_valid_y1, predictions_task1))

print("Task 2 (Koşul) Accuracy:", accuracy_score(bert_valid_y2, predictions_task2))
print("Task 2 (Koşul) Classification Report:")
print(classification_report(bert_valid_y2, predictions_task2))


IndexError: Target 16 is out of bounds.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------
# 1. Adım: Verileri Yükleme
# -----------------------------
train_file = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'
valid_file = r'/content/drive/My Drive/akışlar-test-verileri.xlsx'

df_train = pd.read_excel(train_file)
df_valid = pd.read_excel(valid_file)

# Adım Türüne göre Koşul Label'ı belirle (Koşul varsa 1, yoksa 0)
df_train['Koşul Label'] = df_train['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)
df_valid['Koşul Label'] = df_valid['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)

# Boş olan Evet/Hayır durumlarını -1 yap (Eksik değerler için)
df_train['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_train['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

df_valid['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_valid['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

# Eksik değerleri içermeyenleri filtrele
df_train = df_train[(df_train['Evet Durumu (Sonraki Adım)'] != -1) & (df_train['Hayır Durumu (Sonraki Adım)'] != -1)]
df_valid = df_valid[(df_valid['Evet Durumu (Sonraki Adım)'] != -1) & (df_valid['Hayır Durumu (Sonraki Adım)'] != -1)]

# -----------------------------
# 2. BERT Dataset Sınıfı
# -----------------------------
class BertMultiTaskDataset(Dataset):
    def __init__(self, texts, labels_task1, labels_task2, labels_task3, labels_task4, tokenizer, max_length=128):
        self.texts = texts
        self.labels_task1 = labels_task1
        self.labels_task2 = labels_task2
        self.labels_task3 = labels_task3
        self.labels_task4 = labels_task4
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label_task1 = self.labels_task1[idx]
        label_task2 = self.labels_task2[idx]
        label_task3 = self.labels_task3[idx]
        label_task4 = self.labels_task4[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label_task1': torch.tensor(label_task1, dtype=torch.long),
            'label_task2': torch.tensor(label_task2, dtype=torch.float),
            'label_task3': torch.tensor(label_task3, dtype=torch.long),
            'label_task4': torch.tensor(label_task4, dtype=torch.long),
        }

# -----------------------------
# 3. Model Tanımı
# -----------------------------
class BertMultiTaskModel(nn.Module):
    def __init__(self, bert_model_name, num_labels_task1, num_labels_task3, num_labels_task4):
        super(BertMultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.task1_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task1)
        self.task2_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.task3_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task3)
        self.task4_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task4)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        task1_logits = self.task1_classifier(pooled_output)
        task2_logits = self.task2_classifier(pooled_output).squeeze(-1)
        task3_logits = self.task3_classifier(pooled_output)
        task4_logits = self.task4_classifier(pooled_output)

        return task1_logits, task2_logits, task3_logits, task4_logits

# -----------------------------
# 4. Model Hazırlığı
# -----------------------------
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
num_labels_task1 = len(df_train['Adım Türü Label'].unique())
num_labels_task3 = len(df_train['Evet Durumu (Sonraki Adım)'].unique())
num_labels_task4 = len(df_train['Hayır Durumu (Sonraki Adım)'].unique())

train_dataset = BertMultiTaskDataset(df_train['Adım Açıklaması'].tolist(), df_train['Adım Türü Label'].values, df_train['Koşul Label'].values, df_train['Evet Durumu (Sonraki Adım)'].values, df_train['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)
valid_dataset = BertMultiTaskDataset(df_valid['Adım Açıklaması'].tolist(), df_valid['Adım Türü Label'].values, df_valid['Koşul Label'].values, df_valid['Evet Durumu (Sonraki Adım)'].values, df_valid['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# -----------------------------
# 5. Model Eğitimi
# -----------------------------
model = BertMultiTaskModel("dbmdz/bert-base-turkish-cased", num_labels_task1, num_labels_task3, num_labels_task4)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion_task1 = nn.CrossEntropyLoss()
criterion_task2 = nn.BCEWithLogitsLoss()
criterion_task3 = nn.CrossEntropyLoss(ignore_index=-1)  # -1 değerlerini yok say
criterion_task4 = nn.CrossEntropyLoss(ignore_index=-1)

num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        loss = (
            criterion_task1(task1_logits, batch['label_task1']) +
            criterion_task2(task2_logits, batch['label_task2']) +
            criterion_task3(task3_logits, batch['label_task3']) +
            criterion_task4(task4_logits, batch['label_task4'])
        )
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} tamamlandı!")

# -----------------------------
# 6. Model Değerlendirme
# -----------------------------
model.eval()
predictions_task1, predictions_task2, predictions_task3, predictions_task4 = [], [], [], []
with torch.no_grad():
    for batch in valid_loader:
        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        preds_task1 = torch.argmax(task1_logits, dim=1).tolist()
        preds_task2 = (torch.sigmoid(task2_logits) > 0.5).int().tolist()
        preds_task3 = torch.argmax(task3_logits, dim=1).tolist()
        preds_task4 = torch.argmax(task4_logits, dim=1).tolist()

        predictions_task1.extend(preds_task1)
        predictions_task2.extend(preds_task2)
        predictions_task3.extend(preds_task3)
        predictions_task4.extend(preds_task4)

# Sonuçlar
print("Task 1 (Adım Türü) Accuracy:", accuracy_score(df_valid['Adım Türü Label'], predictions_task1))
print("Task 1 (Adım Türü) Classification Report:")
print(classification_report(df_valid['Adım Türü Label'], predictions_task1))

print("Task 2 (Koşul) Accuracy:", accuracy_score(df_valid['Koşul Label'], predictions_task2))
print("Task 2 (Koşul) Classification Report:")
print(classification_report(df_valid['Koşul Label'], predictions_task2))

print("Task 3 (Evet Durumu) Accuracy:", accuracy_score(df_valid['Evet Durumu (Sonraki Adım)'], predictions_task3))
print("Task 3 (Evet Durumu) Classification Report:")
print(classification_report(df_valid['Evet Durumu (Sonraki Adım)'], predictions_task3))

print("Task 4 (Hayır Durumu) Accuracy:", accuracy_score(df_valid['Hayır Durumu (Sonraki Adım)'], predictions_task4))
print("Task 4 (Hayır Durumu) Classification Report:")
print(classification_report(df_valid['Hayır Durumu (Sonraki Adım)'], predictions_task4))


IndexError: Target 7 is out of bounds.

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# 1. Model Tanımlaması
class BertMultiTaskModel(nn.Module):
    def __init__(self, bert_model_name, num_labels_task1, num_labels_task3, num_labels_task4):
        super(BertMultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.task1_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task1)
        self.task2_classifier = nn.Linear(self.bert.config.hidden_size, 1)  # Koşul (binary classification)
        self.task3_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task3)
        self.task4_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task4)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        task1_logits = self.task1_classifier(pooled_output)
        task2_logits = self.task2_classifier(pooled_output).squeeze(-1)
        task3_logits = self.task3_classifier(pooled_output)
        task4_logits = self.task4_classifier(pooled_output)

        return task1_logits, task2_logits, task3_logits, task4_logits

# 2. Dataset Tanımlaması
class FlowDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['Adım Açıklaması']  # Adım açıklaması
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        labels_task1 = torch.tensor(self.df.iloc[idx]['Adım Türü Label'], dtype=torch.long)
        labels_task2 = torch.tensor(self.df.iloc[idx]['Koşul'], dtype=torch.float)
        labels_task3 = torch.tensor(self.df.iloc[idx]['Evet Durumu (Sonraki Adım)'], dtype=torch.long)
        labels_task4 = torch.tensor(self.df.iloc[idx]['Hayır Durumu (Sonraki Adım)'], dtype=torch.long)

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label_task1': labels_task1,
            'label_task2': labels_task2,
            'label_task3': labels_task3,
            'label_task4': labels_task4
        }

# 3. Kaybı Hesaplamak için Kullanılan Fonksiyonlar
criterion_task1 = nn.CrossEntropyLoss()
criterion_task2 = nn.BCEWithLogitsLoss()
criterion_task3 = nn.CrossEntropyLoss(ignore_index=-1)  # Eksik değerler için yok sayma
criterion_task4 = nn.CrossEntropyLoss(ignore_index=-1)  # Eksik değerler için yok sayma

# 4. Eğitim Parametrelerini Ayarlama
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertMultiTaskModel(bert_model_name='bert-base-uncased', num_labels_task1=10, num_labels_task3=10, num_labels_task4=10)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 5. Veri Yükleyici
train_df = pd.read_csv('train.csv')  # Eğitim verinizi buradan yükleyebilirsiniz
valid_df = pd.read_csv('valid.csv')  # Doğrulama verinizi buradan yükleyebilirsiniz
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  # Maximum length of tokens

train_dataset = FlowDataset(train_df, tokenizer, max_length)
valid_dataset = FlowDataset(valid_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# 6. Eğitim Döngüsü
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        batch = {key: value.to(device) for key, value in batch.items()}

        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        loss = (
            criterion_task1(task1_logits, batch['label_task1']) +
            criterion_task2(task2_logits, batch['label_task2']) +
            criterion_task3(task3_logits, batch['label_task3']) +
            criterion_task4(task4_logits, batch['label_task4'])
        )

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} tamamlandı!")

# 7. Modeli Değerlendirme
model.eval()
predictions_task1, predictions_task2 = [], []

with torch.no_grad():
    for batch in valid_loader:
        batch = {key: value.to(device) for key, value in batch.items()}

        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        preds_task1 = torch.argmax(task1_logits, dim=1).tolist()
        preds_task2 = (torch.sigmoid(task2_logits) > 0.5).int().tolist()

        predictions_task1.extend(preds_task1)
        predictions_task2.extend(preds_task2)

# Doğrulama sonuçları
print("Task 1 (Adım Türü) Accuracy:", accuracy_score(valid_df['Adım Türü Label'], predictions_task1))
print("Task 1 (Adım Türü) Classification Report:")
print(classification_report(valid_df['Adım Türü Label'], predictions_task1))

print("Task 2 (Koşul) Accuracy:", accuracy_score(valid_df['Koşul'], predictions_task2))
print("Task 2 (Koşul) Classification Report:")
print(classification_report(valid_df['Koşul'], predictions_task2))

# 8. Etiket Uyumunu Kontrol Etme
max_label_task1 = train_df['Adım Türü Label'].max()
max_label_task3 = train_df['Evet Durumu (Sonraki Adım)'].max()
max_label_task4 = train_df['Hayır Durumu (Sonraki Adım)'].max()

num_labels_task1 = len(train_df['Adım Türü Label'].unique())
num_labels_task3 = len(train_df['Evet Durumu (Sonraki Adım)'].unique())
num_labels_task4 = len(train_df['Hayır Durumu (Sonraki Adım)'].unique())

assert max_label_task1 < num_labels_task1, f"Task1: Etiketler num_labels_task1'dan büyük! ({max_label_task1} / {num_labels_task1})"
assert max_label_task3 < num_labels_task3, f"Task3: Etiketler num_labels_task3'dan büyük! ({max_label_task3} / {num_labels_task3})"
assert max_label_task4 < num_labels_task4, f"Task4: Etiketler num_labels_task4'dan büyük! ({max_label_task4} / {num_labels_task4})"


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------
# 1. Adım: Verileri Yükleme
# -----------------------------
train_file = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'
valid_file = r'/content/drive/My Drive/akışlar-test-verileri.xlsx'

df_train = pd.read_excel(train_file)
df_valid = pd.read_excel(valid_file)

# Adım Türüne göre Koşul Label'ı belirle (Koşul varsa 1, yoksa 0)
df_train['Koşul Label'] = df_train['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)
df_valid['Koşul Label'] = df_valid['Adım Türü'].apply(lambda x: 1 if x == "Koşul" else 0)

# Boş olan Evet/Hayır durumlarını -1 yap (Eksik değerler için)
df_train['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_train['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_train['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

df_valid['Evet Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Evet Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)
df_valid['Hayır Durumu (Sonraki Adım)'] = pd.to_numeric(df_valid['Hayır Durumu (Sonraki Adım)'], errors='coerce').fillna(-1).astype(int)

# Eksik değerleri içermeyenleri filtrele
df_train = df_train[(df_train['Evet Durumu (Sonraki Adım)'] != -1) & (df_train['Hayır Durumu (Sonraki Adım)'] != -1)]
df_valid = df_valid[(df_valid['Evet Durumu (Sonraki Adım)'] != -1) & (df_valid['Hayır Durumu (Sonraki Adım)'] != -1)]

# -----------------------------
# 2. BERT Dataset Sınıfı
# -----------------------------
class BertMultiTaskDataset(Dataset):
    def __init__(self, texts, labels_task1, labels_task2, labels_task3, labels_task4, tokenizer, max_length=128):
        self.texts = texts
        self.labels_task1 = labels_task1
        self.labels_task2 = labels_task2
        self.labels_task3 = labels_task3
        self.labels_task4 = labels_task4
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label_task1 = self.labels_task1[idx]
        label_task2 = self.labels_task2[idx]
        label_task3 = self.labels_task3[idx]
        label_task4 = self.labels_task4[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label_task1': torch.tensor(label_task1, dtype=torch.long),
            'label_task2': torch.tensor(label_task2, dtype=torch.float),
            'label_task3': torch.tensor(label_task3, dtype=torch.long),
            'label_task4': torch.tensor(label_task4, dtype=torch.long),
        }

# -----------------------------
# 3. Model Tanımı
# -----------------------------
class BertMultiTaskModel(nn.Module):
    def __init__(self, bert_model_name, num_labels_task1, num_labels_task3, num_labels_task4):
        super(BertMultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.task1_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task1)
        self.task2_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.task3_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task3)
        self.task4_classifier = nn.Linear(self.bert.config.hidden_size, num_labels_task4)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        task1_logits = self.task1_classifier(pooled_output)
        task2_logits = self.task2_classifier(pooled_output).squeeze(-1)
        task3_logits = self.task3_classifier(pooled_output)
        task4_logits = self.task4_classifier(pooled_output)

        return task1_logits, task2_logits, task3_logits, task4_logits

# -----------------------------
# 4. Model Hazırlığı
# -----------------------------
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
num_labels_task1 = len(df_train['Adım Türü Label'].unique())
num_labels_task3 = len(df_train['Evet Durumu (Sonraki Adım)'].unique())
num_labels_task4 = len(df_train['Hayır Durumu (Sonraki Adım)'].unique())

train_dataset = BertMultiTaskDataset(df_train['Adım Açıklaması'].tolist(), df_train['Adım Türü Label'].values, df_train['Koşul Label'].values, df_train['Evet Durumu (Sonraki Adım)'].values, df_train['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)
valid_dataset = BertMultiTaskDataset(df_valid['Adım Açıklaması'].tolist(), df_valid['Adım Türü Label'].values, df_valid['Koşul Label'].values, df_valid['Evet Durumu (Sonraki Adım)'].values, df_valid['Hayır Durumu (Sonraki Adım)'].values, bert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# -----------------------------
# 5. Model Eğitimi
# -----------------------------
model = BertMultiTaskModel("dbmdz/bert-base-turkish-cased", num_labels_task1, num_labels_task3, num_labels_task4)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion_task1 = nn.CrossEntropyLoss()
criterion_task2 = nn.BCEWithLogitsLoss()
criterion_task3 = nn.CrossEntropyLoss(ignore_index=-1)  # -1 değerlerini yok say
criterion_task4 = nn.CrossEntropyLoss(ignore_index=-1)

num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        loss = (
            criterion_task1(task1_logits, batch['label_task1']) +
            criterion_task2(task2_logits, batch['label_task2']) +
            criterion_task3(task3_logits, batch['label_task3']) +
            criterion_task4(task4_logits, batch['label_task4'])
        )
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} tamamlandı!")

# -----------------------------
# 6. Model Değerlendirmesi
# -----------------------------
model.eval()
predictions_task1, predictions_task2 = [], []

with torch.no_grad():
    for batch in valid_loader:
        task1_logits, task2_logits, task3_logits, task4_logits = model(batch['input_ids'], batch['attention_mask'])

        preds_task1 = torch.argmax(task1_logits, dim=1).tolist()
        preds_task2 = (torch.sigmoid(task2_logits) > 0.5).int().tolist()

        predictions_task1.extend(preds_task1)
        predictions_task2.extend(preds_task2)

# Doğrulama sonuçları
print("Task 1 (Adım Türü) Accuracy:", accuracy_score(df_valid['Adım Türü Label'], predictions_task1))
print("Task 1 (Adım Türü) Classification Report:")
print(classification_report(df_valid['Adım Türü Label'], predictions_task1))

print("Task 2 (Koşul) Accuracy:", accuracy_score(df_valid['Koşul Label'], predictions_task2))
print("Task 2 (Koşul) Classification Report:")
print(classification_report(df_valid['Koşul Label'], predictions_task2))


IndexError: Target 7 is out of bounds.